In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [ ]:
train_dataset = torchvision.datasets.MNIST(root = "./data", 
                                          train =True,
                                          transform = transforms.ToTensor(),
                                          download = True)

test_dataset = torchvision.datasets.MNIST(root = "./data", 
                                          train = False,
                                          transform = transforms.ToTensor())

In [ ]:
batch_size = 100

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
#         self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first = True, nonlinearity = "relu")
#         self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first = True)
        self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first= True)
        
#         for name, param in self.gru.named_parameters():
#             if "bias" in name:
#                 nn.init.constant_(param, 0.0)
#             elif "weight_ih" in name:
#                 nn.init.kaiming_normal_(param)
#             elif "weight_hh" in name:
#                 nn.init.kaiming_normal_(param)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
#         c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
        out, hn = self.gru(x, h0)
#         out, hn = self.lstm(x, (h0,c0))
        
        out = self.fc(out[:,-1,:])
        
        return out

In [ ]:
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10
seq_len = 28

In [ ]:
model = RNN(input_dim, hidden_dim, layer_dim, output_dim).to(device)

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
total_steps = len(train_loader)
num_epochs = 5

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, seq_len, input_dim).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        l = loss(outputs, labels)
        
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1) % 100 == 0:
            print(f" Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss {l.item():.4f}")

In [ ]:
n_samples =0
n_correct =0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.reshape(-1, seq_len, input_dim).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
    
    print(f"Accuracy : {(n_correct/n_samples)*100} %")